In [1]:
import os 
os.chdir("../..")
from MSML.src.math_spec_mapping import (load_from_json, create_action_chains_graph, write_out_boundary_actions,
                      write_out_policies, write_out_mechanisms, load_svg_graphviz, write_basic_report_full,
                      write_action_chain_reports, write_spec_tree, create_parameter_impact_table,
                     write_entity_reports, write_wiring_report, write_entity_markdown_report,
                     write_all_markdown_reports)
os.chdir("Quai-Macro-Model/Spec")

from IPython.display import display, display_html
from copy import deepcopy
from src import math_spec_json
import numpy as np
from pprint import pprint
from simulation import state_base, params_base, vesting_schedule_translate, build_logistic_classifier


#params_base["FP Mine Block Boundary Action"] = "TEST Mine Block Boundary Action"
state_base["K Qi"] = 1
state_base["K Quai"] = 500
state_base["Population Mining Beta Vector"] = np.array([0, .001])

ms = load_from_json(deepcopy(math_spec_json))
msi = ms.build_implementation(params_base)


Placeholder has no control action options


## Source Code for All Used Components

In [2]:
keys = [x.name for x in ms.wiring['Mine Block Wiring'].components_full()]
msi.print_source_code_files(keys)

--------------------Mint Quai Tokens Mechanism--------------------
def mint_quai_token_mechanism(state, params, spaces):
    amount = spaces[0]["Quai"]
    if amount == 0:
        return
    assert amount > 0, "Amount must be positive"
    state["Quai Supply"] += amount





--------------------Increment Block Number Mechanism--------------------
def increment_block_number_mechanism(state, params, spaces):
    state["Block Number"] += 1





--------------------Controller Update Policy--------------------
def reward_ratio_gain(state, params, spaces):
    # To be set to a parameter soon
    k_qi = state["K Qi"]
    alpha = params["Controller Alpha Parameter"]
    D = spaces[0]["Block Difficulty"]
    D = sum(D) / len(D)
    d1 = D
    d2 = log(D, params["Quai Reward Base Parameter"])
    x_d = d1 / d2
    x_b_star = -spaces[1]["Beta"][0] / spaces[1]["Beta"][1]
    k_qi += alpha * (x_d / x_b_star - 1) * k_qi
    spaces = [{"K Qi": k_qi, "K Quai": state["K Quai"]}, spaces[1]]
    return s

In [3]:
state, params = msi.prepare_state_and_params(state_base, params_base,
                                             state_preperation_functions=[vesting_schedule_translate, build_logistic_classifier],
                                             parameter_preperation_functions=[])

spaces = []
spaces = msi.boundary_actions['Mine Block Boundary Action'](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()


spaces = msi.policies["Block Reward Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()

spaces = msi.policies["Mining Payment Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()


spaces = msi.wiring["Mezzanine Mining Wiring"](state, params, spaces)
pprint(spaces)
print()
print()
print()

spaces = msi.wiring["Mining Mechanisms"](state, params, spaces)

1
[{'Aggregate Hashpower': 1000,
  'Blocks to Mine': [{'Difficulty': 5976.036013216132},
                     {'Difficulty': 5868.828701148912},
                     {'Difficulty': 6287.296965770054},
                     {'Difficulty': 5553.904870814811},
                     {'Difficulty': 6718.509783227358},
                     {'Difficulty': 6330.16003198379},
                     {'Difficulty': 6803.063710547227},
                     {'Difficulty': 6456.771978038954},
                     {'Difficulty': 6062.462409601821},
                     {'Difficulty': 6330.898595883109},
                     {'Difficulty': 6096.402986328913},
                     {'Difficulty': 5776.999422733266},
                     {'Difficulty': 6333.298190101022},
                     {'Difficulty': 6000.317978728947},
                     {'Difficulty': 5950.6261936505},
                     {'Difficulty': 6342.673439060661}],
  'Locking Times': [0.25,
                    0.5,
                    0.

IndexError: list index out of range

In [ ]:
assert False, "Open issue to fix up hashpower comparisons with change in model"

AssertionError: Open issue to fix up hashpower comparisons with change in model

# Hashpower Comparisons

The current dummy model multiplies the Qi price by 1000. We display a low, medium and high scenario below to illustrate when hashpower is not enough, just right and too high and how difficulty adjusts as well as what the spaces look like.

The target mining time is set to 2 seconds.

## Low Hashpower

You can see that a prime block is not mined in the first epoch leading to a decrease in difficulty and two total epochs.

In [ ]:
np.random.seed(0)
state_i = deepcopy(state_base)
state_i["Qi Price"] = 5
state, params = msi.prepare_state_and_params(state_i, params_base, state_preperation_functions=[vesting_schedule_translate], parameter_preperation_functions=[])
spaces = msi.boundary_actions['Mine Block Boundary Action'](state, params, [])
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Block Reward Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Payment Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()

[{'Aggregate Hashpower': 5000,
  'Block Hash Cumulative Sum': array([ 1638,  3168,  6318,  7996,  9643, 12500, 18792]),
  'Blocks to Mine': [{'Block Type': 'Zone', 'Difficulty': 1638},
                     {'Block Type': 'Zone', 'Difficulty': 1530},
                     {'Block Type': 'Region', 'Difficulty': 3150},
                     {'Block Type': 'Zone', 'Difficulty': 1678},
                     {'Block Type': 'Zone', 'Difficulty': 1647},
                     {'Block Type': 'Region', 'Difficulty': 2857},
                     {'Block Type': 'Prime', 'Difficulty': 6292}]}]



[{'Mining Epochs': [{'Mined Blocks': [{'Block Type': 'Zone',
                                       'Difficulty': 1638},
                                      {'Block Type': 'Zone',
                                       'Difficulty': 1530},
                                      {'Block Type': 'Region',
                                       'Difficulty': 3150},
                                      {'Block Type

## Medium Hashpower

One mining epoch happens and within a reasonable time so no difficulty adjustment

In [ ]:
np.random.seed(0)
state_i = deepcopy(state_base)
state_i["Qi Price"] = 10
state, params = msi.prepare_state_and_params(state_i, params_base, state_preperation_functions=[vesting_schedule_translate], parameter_preperation_functions=[])
spaces = msi.boundary_actions['Mine Block Boundary Action'](state, params, [])
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Block Reward Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Payment Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()

[{'Aggregate Hashpower': 10000,
  'Block Hash Cumulative Sum': array([ 1638,  3168,  6318,  7996,  9643, 12500, 18792]),
  'Blocks to Mine': [{'Block Type': 'Zone', 'Difficulty': 1638},
                     {'Block Type': 'Zone', 'Difficulty': 1530},
                     {'Block Type': 'Region', 'Difficulty': 3150},
                     {'Block Type': 'Zone', 'Difficulty': 1678},
                     {'Block Type': 'Zone', 'Difficulty': 1647},
                     {'Block Type': 'Region', 'Difficulty': 2857},
                     {'Block Type': 'Prime', 'Difficulty': 6292}]}]



[{'Mining Epochs': [{'Mined Blocks': [{'Block Type': 'Zone',
                                       'Difficulty': 1638},
                                      {'Block Type': 'Zone',
                                       'Difficulty': 1530},
                                      {'Block Type': 'Region',
                                       'Difficulty': 3150},
                                      {'Block Typ

## High Hashpower

One mining epoch happens but too fast so difficulty is adjusted to be higher

In [ ]:
np.random.seed(0)
state_i = deepcopy(state_base)
state_i["Qi Price"] = 100
state, params = msi.prepare_state_and_params(state_i, params_base, state_preperation_functions=[vesting_schedule_translate], parameter_preperation_functions=[])
spaces = msi.boundary_actions['Mine Block Boundary Action'](state, params, [])
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Block Reward Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Payment Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()

[{'Aggregate Hashpower': 100000,
  'Block Hash Cumulative Sum': array([ 1638,  3168,  6318,  7996,  9643, 12500, 18792]),
  'Blocks to Mine': [{'Block Type': 'Zone', 'Difficulty': 1638},
                     {'Block Type': 'Zone', 'Difficulty': 1530},
                     {'Block Type': 'Region', 'Difficulty': 3150},
                     {'Block Type': 'Zone', 'Difficulty': 1678},
                     {'Block Type': 'Zone', 'Difficulty': 1647},
                     {'Block Type': 'Region', 'Difficulty': 2857},
                     {'Block Type': 'Prime', 'Difficulty': 6292}]}]



[{'Mining Epochs': [{'Mined Blocks': [{'Block Type': 'Zone',
                                       'Difficulty': 1638},
                                      {'Block Type': 'Zone',
                                       'Difficulty': 1530},
                                      {'Block Type': 'Region',
                                       'Difficulty': 3150},
                                      {'Block Ty